# Send gRPC and API Calls via Python Scripts to OpenVINO Model Server in OpenShift 

We will show you how to deploy an OpenVINO Model Server (OVMS) service in an OpenShift cluster. Then, we will send gRPC and REST API calls to the AI inference service by invoking Python scripts.

Requirements:
- OpenShift cluster with the API access to a project
- installed [OpenVINO Model Server Operator](https://catalog.redhat.com/software/operators/search?q=openvino)
- JupyterLab environment with Python3 deployed in the cluster

If you don't have an OpenShift account, you can sign up for 30 or 60 day [free trial of Red Hat OpenShift](https://www.openshift.com/try).

## Login to OpenShift with API Token

First, let's login to OpenShift cluster using `oc` tool. 

In the Red Hat OpenShift console, click on your username and select `Copy login command`.

![copy-login.png](notebook-files/copy-login.png)

Click on `Display Token` and your API token will appear.

![log-in-with-token.png](notebook-files/log-in-with-token.png)

Copy `Log in with token` command and paste it in the cell below. The command has your `<user-API-token>` and `<cluster-DNS-name>`.

In [1]:
!oc login --token=<user-API-token> --server=https://api.<cluster-DNS-name>:6443

Logged into "https://api.openvino5.3q12.p1.openshiftapps.com:6443" as "nyurguhun" using the token provided.

You have access to the following projects and can switch between them with 'oc project <projectname>':

  * ovms
    rose-jh

Using project "ovms".


Create `ovms` project and go to this project.

In [2]:
!oc new-project ovms
!oc project ovms

Error from server (AlreadyExists): project.project.openshift.io "ovms" already exists
Already on project "ovms" on server "https://api.openvino5.3q12.p1.openshiftapps.com:6443".


## Deploy an OVMS Service


Here's the yaml used to configure the OVMS service. We specified `name` to be `ovms-resnet` and `model_path` to be `gs://ovms-public-eu/resnet50-binary`. Also, we defined `model_name` here; we will use this value for gRPC and REST API calls.

In [3]:
!cat ovms.yaml

apiVersion: intel.com/v1alpha1
kind: Ovms
metadata:
  name: ovms-resnet
spec:
  grpc_port: 8080
  image_name: registry.connect.redhat.com/intel/openvino-model-server:latest
  log_level: INFO
  model_name: "resnet"
  model_path: "gs://ovms-public-eu/resnet50-binary"
  plugin_config: '{\"CPU_THROUGHPUT_STREAMS\":\"1\"}'
  replicas: 1
  resources:
    limits:
      cpu: 4
      memory: 500Mi
  rest_port: 8081
  service_type: ClusterIP

Run the cell below to create new OVMS service called `ovms-resnet`.

In [4]:
!oc apply -f ovms.yaml

ovms.intel.com/ovms-resnet created


It takes around 15 seconds to create a pod and a service. They should start with `ovms-resnet`.

In [11]:
!oc get pod
!oc get service

NAME                           READY     STATUS    RESTARTS   AGE
ovms-resnet-5fd9cd8b86-bf8t4   1/1       Running   0          21s
NAME          TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)             AGE
ovms-resnet   ClusterIP   172.30.95.44   <none>        8080/TCP,8081/TCP   22s


Check if the service is up and running by making a REST API call.

In [12]:
!curl http://ovms-resnet.ovms.svc.cluster.local:8081/v1/models/resnet

{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": "OK"
   }
  }
 ]
}


The `state` parameter should be `AVAILABLE`. Note `version` value, we will use it for gRPC and REST API calls. 

## gRPC API Calls

In this section, we will make gRPC API calls to the OVMS service using sample scripts.

To run sample scripts, we will need to get the serving metadata. `get_serving_meta.py` script has the following arguments:
* **--grpc_address**  
* **--grpc_port**
* **--model_name**
* **--model_version**

Run the cell to get the metadata.

In [13]:
!python get_serving_meta.py --grpc_address ovms-resnet.ovms.svc.cluster.local \
                            --grpc_port 8080 \
                            --model_name resnet \
                            --model_version 1

2021-05-12 16:25:16.043692: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-05-12 16:25:16.043730: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Getting model metadata for model: resnet
Inputs metadata:
	Input name: 0; shape: [1, 3, 224, 224]; dtype: DT_FLOAT
Outputs metadata:
	Output name: 1463; shape: [1, 1000]; dtype: DT_FLOAT


For the `jpeg_classificaion.py` script, we specified `--input_name` and `--output_name` arguments, from the metadata output. 

Let's see what's in `input_images.txt` file. We will use for `--images_list` argument.

In [14]:
!cat input_images.txt

    images/airliner.jpeg 404
    images/arctic-fox.jpeg 279
    images/bee.jpeg 309
    images/golden_retriever.jpeg 207
    images/gorilla.jpeg 366
    images/magnetic_compass.jpeg 635
    images/peacock.jpeg 84
    images/pelican.jpeg 144
    images/snail.jpeg 113
    images/zebra.jpeg 340


As you can see, `input_images.txt` contains paths to images and [class ids](classes.py).

Run a classification inference on list of images, listed in `input_images.txt`.

In [15]:
!python3 jpeg_classification.py --grpc_address ovms-resnet.ovms.svc.cluster.local \
                                --grpc_port 8080 \
                                --input_name 0 \
                                --output_name 1463 \
                                --images_list input_images.txt

2021-05-12 16:25:27.083866: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-05-12 16:25:27.083904: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

Processing results.
Model name: resnet
Images list file: input_images.txt

Inference Results:

1. Correct classification: airliner
Processing time: 13.62 ms; speed: 73.41 fps

2. Correct classification: Arctic fox, white fox, Alopex lagopus
Processing time: 5.51 ms; speed: 181.52 fps

3. Correct classification: bee
Processing time: 5.04 ms; speed: 198.53 fps

4. Correct classification: golden retriever
Processing time: 5.20 ms; speed: 192.46 fps

5. Correct classification: gorilla, Gorilla gorilla
Processing time: 5.20 ms; speed: 192.42 fps

6. Correct classification: magnetic compass
Processing time: 5.

Let's try another script, `grpc_serving_client.py`. It accepts NumPy array as an input for `--images_numpy_path` argument. We will also add `labels_numpy_path`, so we can see if the OVMS service labeled the objects correctly. 

In [16]:
!python3 grpc_serving_client.py \
        --grpc_address ovms-resnet.ovms.svc.cluster.local \
        --grpc_port 8080 \
        --input_name 0 \
        --output_name 1463 \
        --images_numpy_path imgs.npy \
        --labels_numpy_path lbs.npy \
        --transpose_input False 

2021-05-12 16:25:30.951862: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-05-12 16:25:30.951896: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Image data range: 0.0 : 255.0
Start processing.
	Model name: resnet
	Iterations: 10
	Images numpy path: imgs.npy
	Images in shape: (10, 3, 224, 224)

1. airliner; Correct match.
Processing time: 7.33 ms; speed: 136.35 fps

2. Arctic fox, white fox, Alopex lagopus; Correct match.
Processing time: 5.38 ms; speed: 185.77 fps

3. bee; Correct match.
Processing time: 4.97 ms; speed: 201.37 fps

4. golden retriever; Correct match.
Processing time: 4.65 ms; speed: 215.05 fps

5. gorilla, Gorilla gorilla; Correct match.
Processing time: 4.43 ms; speed: 225.78 fps

6. magnetic compass; Correct match.
Processing ti

## REST API Calls

In this section, we will make REST API calls to the OVMS service using sample scripts. We will use port `8081` as `--rest_port`, as we specified in the `ovms.yaml` when we created the OVMS service.

Run the cell below to get the model status.

In [17]:
!python rest_get_model_status.py --rest_url http://ovms-resnet.ovms.svc.cluster.local \
                                 --rest_port 8081 \
                                 --model_name resnet

{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": "OK"
   }
  }
 ]
}



Run the cell below to get serving metadata.

In [18]:
!python rest_get_serving_meta.py --rest_url http://ovms-resnet.ovms.svc.cluster.local \
                                 --rest_port 8081 \
                                 --model_name resnet

{
 "modelSpec": {
  "name": "resnet",
  "signatureName": "",
  "version": "1"
 },
 "metadata": {
  "signature_def": {
   "@type": "type.googleapis.com/tensorflow.serving.SignatureDefMap",
   "signatureDef": {
    "serving_default": {
     "inputs": {
      "0": {
       "dtype": "DT_FLOAT",
       "tensorShape": {
        "dim": [
         {
          "size": "1",
          "name": ""
         },
         {
          "size": "3",
          "name": ""
         },
         {
          "size": "224",
          "name": ""
         },
         {
          "size": "224",
          "name": ""
         }
        ],
        "unknownRank": false
       },
       "name": "0"
      }
     },
     "outputs": {
      "1463": {
       "dtype": "DT_FLOAT",
       "tensorShape": {
        "dim": [
         {
          "size": "1",
          "name": ""
         },
         {
          "size": "1000",
          "name": ""
         }
        ],
        "unknownRank": false
       },
       "name": "1463"


Let's run `rest_serving_client.py` script. It takes `--images_numpy_path` and `--labels_numpy_path` as an input. The script return the same output as `grpc_serving_client.py`.

In [19]:
!python rest_serving_client.py --rest_url http://ovms-resnet.ovms.svc.cluster.local \
                               --rest_port 8081 \
                               --model_name resnet \
                               --input_name 0 \
                               --output_name 1463 \
                               --images_numpy_path imgs.npy \
                               --labels_numpy_path lbs.npy \
                               --transpose_input False                

Image data range: 0 : 255
Start processing.
Model name: resnet
Iterations: 10
Images numpy path: imgs.npy
Images in shape: (10, 3, 224, 224)


ImageNet Classification Results:

1. airliner; Correct match.
Processing time: 15.76 ms; speed: 63.45 fps

2. Arctic fox, white fox, Alopex lagopus; Correct match.
Processing time: 12.68 ms; speed: 78.89 fps

3. bee; Correct match.
Processing time: 12.44 ms; speed: 80.39 fps

4. golden retriever; Correct match.
Processing time: 12.22 ms; speed: 81.84 fps

5. gorilla, Gorilla gorilla; Correct match.
Processing time: 12.28 ms; speed: 81.41 fps

6. magnetic compass; Correct match.
Processing time: 12.46 ms; speed: 80.26 fps

7. peacock; Correct match.
Processing time: 12.71 ms; speed: 78.70 fps

8. pelican; Correct match.
Processing time: 12.52 ms; speed: 79.85 fps

9. snail; Correct match.
Processing time: 13.27 ms; speed: 75.34 fps

10. zebra; Correct match.
Processing time: 12.37 ms; speed: 80.82 fps


processing time for all iterations
average 

## Cleanup

Let's free up resources.

In [20]:
!oc delete ovms ovms-resnet

ovms.intel.com "ovms-resnet" deleted


## Next Steps

In this notebook, you have learned how to gRPC and REST API calls to the OpenVINO Model Server service by invoking Python scripts. Next, you can explore other OpenShift OVMS notebooks:

- [Deploy Image Classification with OpenVINO Model Server in OpenShift](../401-model-serving-openshift-resnet/ovms-openshift-resnet.ipynb)
- [Face Detection Multi Model OpenVINO Model Server Deployment in OpenShift](../403-model-serving-openshift-face-detection-dag/ovms-openshift-face-detection-dag.ipynb)